# T5.1 Panel de control con PowerBI
## Ralphy Núñez Mercado
Hai que facer un proxecto con PowerBI que teña os seguintes elementosPower BI:

Dous informes (e vista para móbiles e ordenador por cada informe).
Alomenos 2 gráficas ou elementos gráficos ordinarios de PowerBI.
1 Mapa (aínda que sexa con outros datos sen relación).
Dúas orixes de datos, por exemplo un excel e un JSON.
Relacións entre os datos (alomenos algunha que teña sentido).
Un obxecto visual de Python.
Un orixe de datos cun script de Python.
Emprega varios orixes de datos (diferentes) para comparar.

Na práctica debe verse algo de:

Scrapping
PANDAS
Spark-HDFS <-> PowerBi

E ademáis que quede bonito o deseño do informe :)

Deberase facer con PowerBI Desktop e publicarase a app.powerbi.com coa conta @fernandowirtz.com.
Subirase a esta tarefa o arquivo .pbix. Se é moi grande para subir, darase unha ligazón ao arquivo en OneDrive (coa conta @fernandowirtz.com).
Compartirase a ligazón ao informe publicado en app.powerbi.com tamén co profe (pode poñerse no texto da tarefa ou engadir ao membro da organización co email: scj@fernandowirtz.com)

### ⬇️ Instalar las librerías

In [ ]:
!conda install pip -y || true
!conda install -c conda-forge selenium -y || true
!pip install webdriver_manager || true
!conda install pandas -y || true
!conda install sqlalchemy -y || true
!pip install pyodbc -y || true

^C


### 🦎 Instalar gecko Driver

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\ralphy.nunezmercado\\.wdm\\drivers\\geckodriver\\win64\\v0.36.0\\geckodriver.exe'

### ⬇️ Importar las librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import time
import random

### Conectar el driver de Firefox

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import csv
import re

# Configuración del navegador Chrome
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Descomenta si no quieres abrir el navegador visualmente
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")

# Inicializa el navegador
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)



# Ir a la página
url = "https://www.autoscout24.es/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=E&source=homepage_search-mask"
driver.get(url)

main = driver.find_element(By.TAG_NAME, 'main')
articulos = main.find_elements(By.TAG_NAME, 'article')

datos_coches = []
for articulo in articulos:
    # Marca y Modelo
    marca = articulo.get_attribute('data-make') or ''
    modelo = articulo.get_attribute('data-model') or ''

    # Precio
    precio = articulo.get_attribute('data-price') or ''
    if not precio:
        try:
            precio = articulo.find_element(By.CSS_SELECTOR, '[data-testid="regular-price"]').text
            precio = re.sub(r'[^\d]', '', precio)
        except:
            precio = ''

    # KM
    try:
        km_text = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-mileage_road"]').text
        km = re.sub(r'[^\d]', '', km_text)
    except:
        km = ''

    # Año
    try:
        year_text = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-calendar"]').text
        year = year_text.split('/')[-1] if '/' in year_text else year_text
    except:
        year = ''

    # Tipo de combustible
    try:
        combustible = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-gas_pump"]').text
    except:
        combustible = ''

    # CV
    try:
        cv_text = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-speedometer"]').text
        match = re.search(r'\((\d+)\s*CV\)', cv_text)
        cv = match.group(1) if match else ''
    except:
        cv = ''

    # Ubicación (solo ciudad/provincia, sin "Contáctanos en: • ES-...")
    try:
        ubicacion_raw = articulo.find_element(By.CSS_SELECTOR, '[data-testid="sellerinfo-address"]').text
        # Busca la última palabra después del último espacio (normalmente la ciudad/provincia)
        if "Contáctanos en:" in ubicacion_raw:
            # Extrae lo que hay después del último espacio
            ubicacion = ubicacion_raw.split()[-1]
        else:
            ubicacion = ubicacion_raw
    except:
        ubicacion = ''

    # Transmisión
    try:
        transmision = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-transmission"]').text
    except:
        transmision = ''

    datos_coches.append([
        marca.capitalize(), modelo, precio, km, year, combustible, cv, ubicacion, transmision
    ])

driver.quit()

# Escribe el CSV
with open('coches.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        'Marca', 'Modelo', 'Precio', 'KM', 'año',
        'Tipo de combustile', 'CV', 'Ubicacion', 'Trasmision'
    ])
    writer.writerows(datos_coches)

print("¡Datos guardados en coches.csv!")
#driver.quit()

# driver.quit()

¡Datos guardados en coches.csv!
